<a href="https://colab.research.google.com/github/jtuckayo/scratchpad/blob/main/notebooks/guides/getting-started/v2/cohere_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/cohere-developer-experience/blob/main/notebooks/guides/getting-started/v2/tutorial_pt3_v2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chatbots

As its name implies, the Chat endpoint enables developers to build chatbots that can handle conversations. At the core of a conversation is a multi-turn dialog between the user and the chatbot. This requires the chatbot to have the state (or “memory”) of all the previous turns to maintain the state of the conversation.

In this tutorial, you'll learn about:
- Creating a custom preamble
- Creating a single-turn conversation
- Building the conversation memory
- Running a multi-turn conversation
- Viewing the chat history

You'll learn these by building an onboarding assistant for new hires.

## Setup

To get started, first we need to install the `cohere` library and create a Cohere client.

In [4]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.2/250.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.5 MB/s eta 0:00:00


In [8]:
import cohere
from google.colab import userdata

In [9]:
co = cohere.ClientV2(api_key = userdata.get("COHERE_API_KEY")) # Get your free API key: https://dashboard.cohere.com/api-keys

## Creating a custom preamble

A conversation starts with a system message, or a preamble, to help steer a chatbot’s response toward certain characteristics.

For example, if we want the chatbot to adopt a formal style, the preamble can be used to encourage the generation of more business-like and professional responses.

The recommended approach is to use two H2 Markdown headers: "Task and Context" and "Style Guide" in the exact order.

In the example below, the preamble provides context for the assistant's task (task and context) and encourages the generation of rhymes as much as possible (style guide).

In [10]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write a short introduction message to my teammates."

# Create a custom system message
system_message="""## Task and Context
You are an assistant who assist new employees of Co1t with their first week.

## Style Guide
Try to speak in rhymes as much as possible. Be professional."""

# Add the messages
messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": message}]

# Generate the response
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

Here's a little note, a friendly hello,
To introduce yourself, a great way to go:

"Hello, fellow Co1t crew,
My name is [Your Name], new to all of you.
Excited to join this innovative team,
Where ideas bloom and success is supreme.

I'm eager to learn and contribute my part,
Let's connect and create, a fresh start.
Looking forward to meeting you all,
And together, we'll make our vision tall."

A simple rhyme, a friendly gesture,
To begin your journey, a memorable adventure.


Further reading:
- [Documentation on preambles](https://docs.cohere.com/docs/preambles)

## Starting the first conversation turn

Let's start with the first conversation turn.

Here, we are also adding a custom preamble or system message for generating a concise response, just to keep the outputs brief for this tutorial.

In [11]:
# Add the user message
message = "I'm joining a new startup called Co1t today. Could you help me write a short introduction message to my teammates."

# Create a custom system message
system_message="""## Task and Context
Generate concise responses, with maximum one-sentence."""

# Add the messages
messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": message}]

# Generate the response
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hi everyone! I'm thrilled to join Co1t today and look forward to collaborating with this talented team, let's create something amazing together!"


## Building the conversation memory

Now, we want the model to refine the earlier response. This requires the next generation to have access to the state, or memory, of the conversation.

To do this, we append the `messages` with the model's previous response using the `assistant` role.

Next, we also append a new user message (for the second turn) to the `messages` list.

Looking at the response, we see that the model is able to get the context from the chat history. The model is able to capture that "it" in the user message refers to the introduction message it had generated earlier.

In [ ]:
# Append the previous response
messages.append({'role' : 'assistant', 'content': response.message.content[0].text})

# Add the user message
message = "Make it more upbeat and conversational."

# Append the user message
messages.append({"role": "user", "content": message})

# Generate the response with the current chat history as the context
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hey, future Co1t buddies! Stoked to join this awesome team, let's get to know each other and make some startup magic together!"


Further reading:
- [Documentation on using the Chat endpoint](https://docs.cohere.com/docs/chat-api)

## Running a multi-turn conversation


You can continue doing this for any number of turns by continuing to append the chatbot's response and the new user message to the `messages` list.

In [ ]:
# Append the previous response
messages.append({"role": "assistant", "content": response.message.content[0].text})

# Add the user message
message = "Thanks. Could you create another one for my DM to my manager."

# Append the user message
messages.append({"role": "user", "content": message})

# Generate the response with the current chat history as the context
response = co.chat(model="command-r-plus-08-2024",
                   messages=messages)

print(response.message.content[0].text)

"Hi, boss! So excited to dive into my new role at Co1t and eager to learn from your mentorship and guidance. Let's crush it!"


## Viewing the chat history

To look at the current chat history, you can print the `messages` list, which contains a list of `user` and `assistant` turns in the same sequence as they were created.

In [ ]:
# Append the previous response
messages.append({"role": "assistant", "content": response.message.content[0].text})

# View the chat history
for message in messages:
    print(message,"\n")

{'role': 'system', 'content': '## Task and Context\nGenerate concise responses, with maximum one-sentence.'} 

{'role': 'user', 'content': "I'm joining a new startup called Co1t today. Could you help me write a short introduction message to my teammates."} 

{'role': 'assistant', 'content': '"Hello, teammates! I\'m thrilled to join the Co1t family today and looking forward to getting to know you all and contributing to our shared success."'} 

{'role': 'user', 'content': 'Make it more upbeat and conversational.'} 

{'role': 'assistant', 'content': '"Hey, future Co1t buddies! Stoked to join this awesome team, let\'s get to know each other and make some startup magic together!"'} 

{'role': 'user', 'content': 'Thanks. Could you create another one for my DM to my manager.'} 

{'role': 'assistant', 'content': '"Hi, boss! So excited to dive into my new role at Co1t and eager to learn from your mentorship and guidance. Let\'s crush it!"'} 



## Conclusion

In this tutorial, you learned about:
- How to create a custom preamble
- How to create a single-turn conversation
- How to build the conversation memory
- How to run a multi-turn conversation
- How to view the chat history

You will use the same method for running a multi-turn conversation when you learn about other use cases such as RAG (Part 6) and tool use (Part 7).

But to fully leverage these other capabilities, you will need another type of language model that generates text representations, or embeddings.

In Part 4, you will learn how text embeddings can power an important use case for RAG, which is semantic search.